# [주식 산업 분류](https://purplechip.tistory.com/32)
> 한국에서 사용되는 산업 분류 기준이 4가지인데 그것들이 바로 WICS, FICS, GICS, KRX지수이다.

체감이 쉽게 되는 예시를 하나 들어보면 '삼성전자' 에 대한 각 산업분류는 다음과 같다.
- WICS : 반도체와반도체장비
- FICS : 휴대폰 및 관련부품
- GICS : 하드웨어및IT장비
- KRX지수 : 통신 및 방송 장비 제조업 (아마도..)

개인적인 생각으로는 WICS가 가장 타당한 것으로 느껴진다. 그럼 각 산업분류 기준에 대해 알아보자.

In [9]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

# KRX

In [2]:
import os 
import dart_fss as dart 

In [4]:
dart.set_api_key(api_key=os.environ['DART_PSS_API_KEY'])

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.get_corp_list()

print(f'공시된 기업 전체 수: {len(corp_list)}')

공시된 기업 전체 수: 102356


In [7]:
# 코스피 기업정보들
stock_corps = [] 

for corp in corp_list:
    try:
        if corp.corp_cls == "Y" or corp.corp_cls == "K": # 코스피/코스탁 기업정보 KOSPI
            kospi_corp = {
                "corp_code": corp.corp_code if corp.corp_code else None,
                "corp_name": corp.corp_name if corp.corp_name else None,
                "stock_code": corp.stock_code if corp.stock_code else None, 
                "corp_cls": corp.corp_cls if corp.corp_cls else None,
                "corp_cls_nm": "kospi" if corp.corp_cls == "Y" else "kosdaq",
                "sector": corp.sector if corp.sector else None,
                "product": corp.product if corp.product else None
            }

            stock_corps.append(kospi_corp)

    except:
        pass

In [13]:
df = pd.DataFrame(stock_corps)

print(df.shape)
df.head()

(2494, 7)


,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,sector,product
0,00956028,엑세스바이오,950130,K,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)"
1,00783246,글로벌에스엠,900070,K,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)"
2,01170962,GRT,900290,K,kosdaq,기타 금융업,"광학필름, 포장필름"
3,00600013,맵스리얼티1,094800,Y,kospi,신탁업 및 집합투자업,None
4,01139266,로스웰,900260,K,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈"


# WICS

In [10]:
import requests
from datetime import datetime 
from pandas import DataFrame

from bs4 import BeautifulSoup as bs

## WICS 소분류 

In [11]:
def add_industry_wics_ss_nm(stock_code):
    url = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={stock_code}'.format(stock_code=stock_code)
    headers = {
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'Referer':'https://finance.naver.com/item/coinfo.naver?code={stock_code}'.format(stock_code=stock_code)
    }
    response = requests.get(url, headers=headers)

    soup = bs(response.text, "html.parser")
    for node in soup.select(".line-left"):
        if 'WICS' in node.get_text():
            return node.get_text().replace('WICS : ', '')

In [14]:
df['industry_wics_ss_nm'] = df['stock_code'].map(add_industry_wics_ss_nm)

df.shape

(2494, 8)

In [27]:
wics_ss_dic = {
    # 에너지
    '에너지장비및서비스': 101010,
    '석유와가스': 101020,
    # 소재
    '화학': 151010,
    '포장재': 151030,
    '비철금속': 151040,
    '철강': 151050,
    '종이와목재': 151060,
    # 자본재
    '우주항공과국방': 201010,
    '건축제품': 201020,
    '건축자재': 201025,
    '건설': 201030,
    '가구': 201035,
    '전기장비': 201040,
    '복합기업': 201050,
    '기계': 201060,
    '조선': 201065,
    '무역회사와판매업체': 201070,
    # 상업서비스와공급품
    '상업서비스와공급품': 202010,
    # 운송
    '항공화물운송과물류': 203010,
    '항공사': 203020,
    '해운사': 203030,
    '도로와철도운송': 203040,
    '운송인프라': 203050,
    # 자동차와부품
    '자동차부품': 251010,
    '자동차': 251020,
    # 내구소비재와의류
    '가정용기기와용품': 252040,
    '레저용장비와제품': 252050,
    '섬유,의류,신발,호화품': 252060,
    '화장품': 252065,
    '문구류': 252070,
    # 호텔,레스토랑,레저 등
    '호텔,레스토랑,레저': 253010,
    '다각화된소비자서비스': 253020,
    # 소매(유통)
    '판매업체': 255010,
    '인터넷과카탈로그소매': 255020,
    '백화점과일반상점': 255030,
    '전문소매': 255040,
    # 교육서비스
    '교육서비스': 256010,
    # 식품과기본식료품소매
    '식품과기본식료품소매': 301010,
    # 식품,음료,담배
    '음료': 302010,
    '식품': 302020,
    '담배': 302030,
    # 가정용품과개인용품
    '가정용품': 303010,
    # 건강관리장비와서비스
    '건강관리장비와용품': 351010,
    '건강관리업체및서비스': 351020,
    '건강관리기술': 351030,
    # 제약과생물공학
    '생물공학': 352010,
    '제약': 352020,
    '생명과학도구및서비스': 352030,
    # 은행
    '은행': 401010,
    # 증권
    '증권': 402010,
    # 다각화된금융
    '창업투자': 403020,
    '카드': 403030,
    '기타금융': 403040,
    # 보험
    '손해보험': 404010,
    '생명보험': 404020,
    # 부동산
    '부동산': 405020,
    # 소프트웨어와서비스
    'IT서비스': 451020,
    '소프트웨어': 451030,
    # 기술하드웨어와장비
    '통신장비': 452010,
    '핸드셋': 452015,
    '컴퓨터와주변기기': 452020,
    '전자장비와기기': 452030,
    '사무용전자제품': 452040,
    # 반도체와반도체장비
    '반도체와반도체장비': 453010,
    # 전자와 전기제품
    '전자제품': 453510,
    '전기제품': 453520,
    # 디스플레이
    '디스플레이패널': 454010,
    '디스플레이장비및부품': 454020,
    # 전기통신서비스
    '다각화된통신서비스': 501010,
    '무선통신서비스': 501020,
    # 미디어와엔터테인먼트
    '광고': 502010,
    '방송과엔터테인먼트': 502020,
    '출판': 502030,
    '게임엔터테인먼트': 502040,
    '양방향미디어와서비스': 502050,
    # 유틸리티
    '전기유틸리티': 551010,
    '가스유틸리티': 551020,
    '복합유틸리티': 551030,
    '독립전력생산및에너지거래': 551050
    
}

def get_wics_ss_dic(wics_ss_nm):
    try:
        return wics_ss_dic[wics_ss_nm]
    except:
        return 0

In [28]:
df['industry_wics_ss_cd'] = df['industry_wics_ss_nm'].map(get_wics_ss_dic)

df.isnull().sum()

corp_code               0
corp_name               0
stock_code              0
corp_cls                0
corp_cls_nm             0
sector                  0
product                14
industry_wics_ss_nm     0
industry_wics_ss_cd     0
dtype: int64

In [29]:
c= df['industry_wics_ss_cd'].isnull()

df.loc[c]

,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,sector,product,industry_wics_ss_nm,industry_wics_ss_cd


## WICS 중분류 

In [33]:
wics_mc_dict = {
    1010:'에너지',
    1510:'소재',
    2010:'자본재',
    2020:'상업서비스와공급품',
    2030:'운송',
    2510:'자동차와부품',
    2520:'내구소비재와의류',
    2530:'호텔,레스토랑,레저 등',
    2550:'소매(유통)',
    2560:'교육서비스',
    3010:'식품과기본식료품소매',
    3020:'식품,음료,담배',
    3030:'가정용품과개인용품',
    3510:'건강관리장비와서비스',
    3520:'제약과생물공학',
    4010:'은행',
    4020:'증권',
    4030:'다각화된금융',
    4040:'보험',
    4050:'부동산',
    4510:'소프트웨어와서비스',
    4520:'기술하드웨어와장비',
    4530:'반도체와반도체장비',
    4535:'전자와 전기제품',
    4540:'디스플레이',
    5010:'전기통신서비스',
    5020:'미디어와엔터테인먼트',
    5510:'유틸리티'}
    
def get_wics_mc_dic(wics_mc_cd):
    try:
        return wics_mc_dict[wics_mc_cd]
    except:
        return None

In [34]:
df['industry_wics_ms_cd'] = df['industry_wics_ss_cd'].map(lambda x: int(str(x)[:4]) if x > 0 else x)
df['industry_wics_ms_nm'] = df['industry_wics_ms_cd'].map(get_wics_mc_dic)

print(df.isnull().sum())
df.head()

corp_code               0
corp_name               0
stock_code              0
corp_cls                0
corp_cls_nm             0
sector                  0
product                14
industry_wics_ss_nm     0
industry_wics_ss_cd     0
industry_wics_ms_cd     0
industry_wics_ms_nm     3
dtype: int64


,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,sector,product,industry_wics_ss_nm,industry_wics_ss_cd,industry_wics_ms_cd,industry_wics_ms_nm
0,00956028,엑세스바이오,950130,K,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)",생명과학도구및서비스,352030,3520,제약과생물공학
1,00783246,글로벌에스엠,900070,K,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)",자동차부품,251010,2510,자동차와부품
2,01170962,GRT,900290,K,kosdaq,기타 금융업,"광학필름, 포장필름",화학,151010,1510,소재
3,00600013,맵스리얼티1,094800,Y,kospi,신탁업 및 집합투자업,None,미분류,0,0,None
4,01139266,로스웰,900260,K,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈",자동차부품,251010,2510,자동차와부품


## WICS 대분류 

In [35]:
wics_lc_dict = {
    10:'에너지', 
    15:'소재', 
    20:'산업재', 
    25:'경기관련소비재', 
    30:'필수소비재', 
    35:'건강관리',
    40:'금융', 
    45:'IT', 
    50:'커뮤니케이션서비스', 
    55:'유틸리티'}

def get_wics_lc_dict(wics_lc_cd):
    try:
        return wics_lc_dict[wics_lc_cd]
    except:
        return None

In [36]:
df['industry_wics_ls_cd'] = df['industry_wics_ms_cd'].map(lambda x: int(str(x)[:2]) if x > 0 else x)
df['industry_wics_ls_nm'] = df['industry_wics_ls_cd'].map(get_wics_lc_dict)

print(df.isnull().sum())
df.head()

corp_code               0
corp_name               0
stock_code              0
corp_cls                0
corp_cls_nm             0
sector                  0
product                14
industry_wics_ss_nm     0
industry_wics_ss_cd     0
industry_wics_ms_cd     0
industry_wics_ms_nm     3
industry_wics_ls_cd     0
industry_wics_ls_nm     3
dtype: int64


,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,sector,product,industry_wics_ss_nm,industry_wics_ss_cd,industry_wics_ms_cd,industry_wics_ms_nm,industry_wics_ls_cd,industry_wics_ls_nm
0,00956028,엑세스바이오,950130,K,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)",생명과학도구및서비스,352030,3520,제약과생물공학,35,건강관리
1,00783246,글로벌에스엠,900070,K,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)",자동차부품,251010,2510,자동차와부품,25,경기관련소비재
2,01170962,GRT,900290,K,kosdaq,기타 금융업,"광학필름, 포장필름",화학,151010,1510,소재,15,소재
3,00600013,맵스리얼티1,094800,Y,kospi,신탁업 및 집합투자업,None,미분류,0,0,None,0,None
4,01139266,로스웰,900260,K,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈",자동차부품,251010,2510,자동차와부품,25,경기관련소비재


# 메타 분석

## 결측치

In [51]:
df.isnull().sum()

corp_code               0
corp_name               0
stock_code              0
corp_cls                0
corp_cls_nm             0
sector                  0
product                14
industry_wics_ss_nm     0
industry_wics_ss_cd     0
industry_wics_ms_cd     0
industry_wics_ms_nm     3
industry_wics_ls_cd     0
industry_wics_ls_nm     3
dtype: int64

In [52]:
c= df['industry_wics_ms_nm'].isnull()

df.loc[c]

,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,sector,product,industry_wics_ss_nm,industry_wics_ss_cd,industry_wics_ms_cd,industry_wics_ms_nm,industry_wics_ls_cd,industry_wics_ls_nm
3,00600013,맵스리얼티1,094800,Y,kospi,신탁업 및 집합투자업,None,미분류,0,0,None,0,None
2033,00907013,한국ANKOR유전,152550,Y,kospi,신탁업 및 집합투자업,None,미분류,0,0,None,0,None
2034,00960641,한국패러랠,168490,Y,kospi,신탁업 및 집합투자업,None,미분류,0,0,None,0,None


## 회사 수

In [38]:
df['stock_code'].nunique() # 전체 회사 수

2494

- 코스탁 / 코스피

In [39]:
df.groupby(['corp_cls_nm'])['stock_code'].agg('nunique')

corp_cls_nm
kosdaq    1658
kospi      836
Name: stock_code, dtype: int64

- sector

In [44]:
df.groupby(['sector'])['stock_code'].agg('nunique').sort_values(ascending=False)

sector
소프트웨어 개발 및 공급업     161
특수 목적용 기계 제조업      154
전자부품 제조업           122
의약품 제조업             98
금융 지원 서비스업          98
                  ... 
시장조사 및 여론조사업         1
신발 및 신발 부분품 제조업      1
재 보험업                1
기타 생활용품 소매업          1
기타 전문 서비스업           1
Name: stock_code, Length: 158, dtype: int64

- industry_wics_ss_nm

In [46]:
df.groupby(['industry_wics_ss_nm'])['stock_code'].agg('nunique').sort_values(ascending=False)

industry_wics_ss_nm
제약            156
반도체와반도체장비     145
자동차부품         141
화학            109
창업투자           97
             ... 
복합유틸리티          1
담배              1
다각화된소비자서비스      1
카드              1
사무용전자제품         1
Name: stock_code, Length: 79, dtype: int64

- industry_wics_ms_nm

In [47]:
df.groupby(['industry_wics_ms_nm'])['stock_code'].agg('nunique').sort_values(ascending=False)

industry_wics_ms_nm
자본재             308
소재              249
제약과생물공학         239
기술하드웨어와장비       230
내구소비재와의류        164
소프트웨어와서비스       153
자동차와부품          146
반도체와반도체장비       145
미디어와엔터테인먼트      123
다각화된금융          105
식품,음료,담배         90
건강관리장비와서비스       89
디스플레이            85
전자와 전기제품         65
운송               44
소매(유통)           42
에너지              37
부동산              27
상업서비스와공급품        26
증권               25
호텔,레스토랑,레저 등     18
교육서비스            18
유틸리티             17
보험               13
은행               11
식품과기본식료품소매        8
가정용품과개인용품         8
전기통신서비스           6
Name: stock_code, dtype: int64

- industry_wics_ls_nm

In [49]:
df.groupby(['industry_wics_ls_nm'])['stock_code'].agg('nunique').sort_values(ascending=False)

industry_wics_ls_nm
IT           678
경기관련소비재      388
산업재          378
건강관리         328
소재           249
금융           181
커뮤니케이션서비스    129
필수소비재        106
에너지           37
유틸리티          17
Name: stock_code, dtype: int64

In [45]:
df.head()

,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,sector,product,industry_wics_ss_nm,industry_wics_ss_cd,industry_wics_ms_cd,industry_wics_ms_nm,industry_wics_ls_cd,industry_wics_ls_nm
0,00956028,엑세스바이오,950130,K,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)",생명과학도구및서비스,352030,3520,제약과생물공학,35,건강관리
1,00783246,글로벌에스엠,900070,K,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)",자동차부품,251010,2510,자동차와부품,25,경기관련소비재
2,01170962,GRT,900290,K,kosdaq,기타 금융업,"광학필름, 포장필름",화학,151010,1510,소재,15,소재
3,00600013,맵스리얼티1,094800,Y,kospi,신탁업 및 집합투자업,None,미분류,0,0,None,0,None
4,01139266,로스웰,900260,K,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈",자동차부품,251010,2510,자동차와부품,25,경기관련소비재


# 저장 

In [50]:
save_path_wics = '../data/meta/stock_corps.csv'

df.to_csv(save_path_wics, header=True, index=False, encoding='utf-8')